### 1번

In [47]:
# input 정의 + padding, filter 정의

import numpy as np
inputs = np.array([[2,0,2,1,3,2],[0,2,0,2,2,2],[1,0,1,3,1,1],[0,0,1,1,1,0],[0,1,3,4,1,0],[0,1,0,0,5,2]])
inputs_pad = np.pad(inputs, pad_width = 1, mode = 'constant', constant_values = 0)
filter_ = np.array([[1,0,0],[0,1,0],[0,0,1]])

In [51]:
# convolution

row = inputs_pad.shape[0] - filter_.shape[0] + 1
col = inputs_pad.shape[1] - filter_.shape[1] + 1

feature_map = []

for rn in range(row):
    for cn in range(col):
        result1 = inputs_pad[rn:rn+filter_.shape[0], cn:cn+filter_.shape[1]] * filter_
        feature_map.append(np.sum(result1))

feature_map = np.array(result).reshape(row,col)
feature_map

array([[ 4,  0,  4,  3,  5,  2],
       [ 0,  5,  3,  5,  4,  5],
       [ 1,  1,  4,  4,  3,  3],
       [ 1,  4,  5,  3,  4,  1],
       [ 1,  1,  3, 10,  4,  1],
       [ 0,  1,  1,  3,  9,  3]])

In [74]:
# max pooling 하는 함수 정의

def max_pooling(x, stride):
    mp =[]
    for i in range(0,len(x)-1,stride):
        for j in range(0,len(x)-1,stride):
            mp.append(np.max(x[i:i+stride, j:j+stride]))
    
    mp = np.array(mp)
    return mp.reshape(stride+1, stride+1)

In [75]:
# max pooling 결과

max_pooling(feature_map,2)

array([[ 5,  5,  5],
       [ 4,  5,  4],
       [ 1, 10,  9]])

## 2번 기존 코드

In [16]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [17]:
# GPU 사용설정, seed 설정

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [18]:
# parameter 설정

learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [19]:
# dataset 가져오기

mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = False)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = False)

In [20]:
# dataset loader 

data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                     batch_size = batch_size,
                                     shuffle = True,
                                     drop_last = True)

In [21]:
# CNN model 

class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
        
    
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
            
        self.fc1 = torch.nn.Linear(4*4*128, 625 ,bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p = 1 - self.keep_prob))
            
        self.fc2 = torch.nn.Linear(625, 10 ,bias = True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out



In [22]:
model = CNN().to(device)

In [23]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [11]:
# 모델 학습

total_batch = len(data_loader)
print ('시간이 좀 걸림')
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
print('Learning finished')

시간이 좀 걸림
[Epoch:    1] cost = 0.18319194
[Epoch:    2] cost = 0.0484953299
[Epoch:    3] cost = 0.0339777395
[Epoch:    4] cost = 0.0265932828
[Epoch:    5] cost = 0.0222814288
[Epoch:    6] cost = 0.0177897308
[Epoch:    7] cost = 0.0158942658
[Epoch:    8] cost = 0.0126739861
[Epoch:    9] cost = 0.0135673871
[Epoch:   10] cost = 0.0113147469
[Epoch:   11] cost = 0.00921845715
[Epoch:   12] cost = 0.00931776781
[Epoch:   13] cost = 0.00696808565
[Epoch:   14] cost = 0.00651066517
[Epoch:   15] cost = 0.00822876394
Learning finished


In [12]:
# 모델 성능 확인

with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test =  mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\Users\Hyuk\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\Hyuk\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9661999940872192


## 2번

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
# GPU 사용설정, seed 설정

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameter 설정

learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# dataset 가져오기

mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = False)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = False)

In [5]:
# dataset loader 

data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                     batch_size = batch_size,
                                     shuffle = True,
                                     drop_last = True)

In [10]:
# CNN model 

class CNN2(torch.nn.Module):
    
    def __init__(self):
        super(CNN2, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size = 4, stride = 1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
 
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 27, 27, 32)
        #    Pool     -> (?, 13, 13, 32)
         
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
 
        # L2 ImgIn shape=(?, 13, 13, 32)
        #    Conv      ->(?, 13, 13, 64)
        #    Pool      ->(?, 6, 6, 64)
 
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 2, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))

        # L3 ImgIn shape=(?, 6, 6, 64)
        #    Conv      ->(?, 3, 3, 128)
        #    Pool      ->(?, 2, 2, 128)
         
        self.fc1 = torch.nn.Linear(2*2*128, 625 ,bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p = 1 - self.keep_prob))
            
        self.fc2 = torch.nn.Linear(625, 10 ,bias = True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out



#### 2-2) layer 3 의 output size = (2,2,128)

In [11]:
model2 = CNN2().to(device)

In [12]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr= learning_rate)

In [13]:
# 모델 학습

total_batch = len(data_loader)
print ('시간이 좀 걸림')
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model2(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
print('Learning finished')

시간이 좀 걸림
[Epoch:    1] cost = 0.265460581
[Epoch:    2] cost = 0.0694887415
[Epoch:    3] cost = 0.0464758128
[Epoch:    4] cost = 0.0377755277
[Epoch:    5] cost = 0.0294301193
[Epoch:    6] cost = 0.0255783852
[Epoch:    7] cost = 0.02240525
[Epoch:    8] cost = 0.0184323676
[Epoch:    9] cost = 0.01873108
[Epoch:   10] cost = 0.0142349014
[Epoch:   11] cost = 0.0135582378
[Epoch:   12] cost = 0.0126877613
[Epoch:   13] cost = 0.0114614824
[Epoch:   14] cost = 0.0087939417
[Epoch:   15] cost = 0.0100883404
Learning finished


In [15]:
# 모델 성능 확인

with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test =  mnist_test.test_labels.to(device)
    
    prediction = model2(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9783999919891357


#### 2-3) 기존 CNN 모델에 비해 cost가 소폭 상승, accuracy도 소폭 상승 했지만 거의 비슷